In [1]:
# 성능개선1. 베이스라인 모델을 그대로 사용하되, 피처 엔지니어링과 하이퍼파라미터 최적화 적용
# + 파생 피처 추가

In [2]:
import pandas as pd
# 모델링 전략
# 고급 모델링 기법 : OOF 예측, 베이지안 최적화, LightGBM, XGBoost 앙상블

# 1. BaseLine : LightGBM, 마이크로소프트 ~ XGBoost와 함계 캐글에서 가장 많이 사용하는 머신러닝 모델
# 훈련과 예측 과정이 동시에 이루어진다.

# 데이터 불러오기
data_path = 'data/'

# train
train = pd.read_csv(data_path + 'train.csv', index_col = 'id')

# test
test = pd.read_csv(data_path + 'test.csv', index_col = 'id')

# sample 결과
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col = 'id')

In [3]:
# 피처 엔지니어링
# 머신 러닝 알고리즘의 성능을 향상하기 위해 데이터를 변환하고 개선하는 프로세스

# 데이터 합치기 : 두 데이터를 동일한 인코딩을 적용하기 위함.
all_data = pd.concat([train, test], ignore_index = True) # 기존 인덱스를 무시하며 train, test 합침.
all_data = all_data.drop('target', axis = 1) # 타겟값 제거, 레이블 축 : 세로 이므로 1.

# 피처만 떼어놓기
all_features = all_data.columns

In [4]:
# 명목형 피처 원-핫 인코딩
# 고윳값별 순서가 없기 때문에 적용가능.
# 복잡한 데이터를 그대로 사용하지 않고 컴퓨터가 처리하기 쉽게 숫자로 변형해 줌.
# 범주형 데이터가 순서나 크기의 의미를 포함하고 있을 때 원핫 인코딩을 하게되면 
# 이러한 정보들은 사용할 수 없게된다는 것입니다. 
# 예를 들면 월요일, 화요일, 수요일, 목요일, 금요일, 토요일, 일요일을 원핫 인코딩하면 
# 그저 0과 1일 집합으로 표현됩니다. 
# e.g. 월요일은 [1,0,0,0,0,0,0], 토요일은 [0,0,0,0,0,1,0], 일요일은 [0,0,0,0,0,0,1] 이렇게 되겠지요. 
# 이렇게 인코딩된 값들은 요일간에 순서나 크기를 표현하지 못합니다.

from sklearn.preprocessing import OneHotEncoder

# 명목형 피처 추출
cat_features = [feature for feature in all_features if 'cat' in feature] # cat이 포함된 feature만 뽑아냄.

onehot_encoder = OneHotEncoder() # 원-핫 인코더 객체 생성

# 인코딩
encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

In [5]:
# 파생 피처 추가 - 성능 향상을 위한 방법들
# 1. 한 데이터가 가진 결측값을 파생 피처로 만든다.

# '데이터 하나당 결측값 개수'를 파생 피처로 추가
all_data['num_missing'] = (all_data==-1).sum(axis=1) # y축으로 데이터 추가하므로 axis = 1

# 결측값 개수를 num_missing이라는 피처명으로 all_data에 추가함. -> 각 데이터 피쳐벌로 결측값을 전부 더한다.

In [6]:
# 명목형 피처, calc 분류의 피처를 제외한 피처

# 1. 원핫 인코딩 적용한 피처 : 명목형 피처
remaining_features = [feature for feature in all_features
                     if('cat' not in feature and # 명목형 피처
                        'calc' not in feature)]

# num_missing을 remaining_features에 추가 - 파생 피처 추가
remaining_features.append('num_missing')

In [7]:
# 2. ind 분류의 피처를 연결해서 새로운 파생 피처로 추가 : mix_ind
# 분류가 ind인 피처
ind_features = [feature for feature in all_features if 'ind' in feature] # ind인 모든 피처를 가져온다.

is_first_feature = True # 첫번째인지 판별하기 위한 flag
for ind_feature in ind_features:
    if is_first_feature:
        all_data['mix_ind'] = all_data[ind_feature].astype(str) + '_' # feature의 데이터 값을 str로 형변환 + '_' : 선언
        is_first_feature = False # flag 제거
    else:
        all_data['mix_ind'] += all_data[ind_feature].astype(str) +'_'
        

# ind 분류의 모든 피처 값을 연결한 파생 피처

In [8]:
all_data['mix_ind'].head()

0    2_2_5_1_0_0_1_0_0_0_0_0_0_0_11_0_1_0_
1     1_1_7_0_0_0_0_1_0_0_0_0_0_0_3_0_0_1_
2    5_4_9_1_0_0_0_1_0_0_0_0_0_0_12_1_0_0_
3     0_1_2_0_0_1_0_0_0_0_0_0_0_0_8_1_0_0_
4     0_2_0_1_0_1_0_0_0_0_0_0_0_0_9_1_0_0_
Name: mix_ind, dtype: object

In [9]:
# 3. 명목형 피처의 고윳값 개수를 새로운 피처로 추가한다.
# 피처의 고유값 각각의 개수는 value_counts()로 구할 수 있다.

all_data['ps_ind_02_cat'].value_counts().to_dict()

# 해당 피처 내에서, 고윳값 1, 고윳값 2 등등의 개수


{1: 1079327, 2: 309747, 3: 70172, 4: 28259, -1: 523}

In [10]:
# 명목형 피처의 고윳값에 대한 파생 피처
cat_count_features = []

for feature in cat_features + ['mix_ind']: # c.f. mix_ind 피처도 명목형 피처이므로, 같이 추가해준다.
    val_counts_dict = all_data[feature].value_counts().to_dict() #각 고윳값을 key로 갖는 dict
    all_data[f'{feature}_count'] = all_data[feature].apply(lambda x : val_counts_dict[x])
    # all_data['@@_count'] = all_data[feature]의 모든 데이터에 고유값을 고유값의 개수로 치환한다.
    # apply : mapping 함수.
    
    cat_count_features.append(f'{feature}_count')

In [11]:
all_data['ps_ind_02_cat'].head()

0    2
1    1
2    4
3    1
4    2
Name: ps_ind_02_cat, dtype: int64

In [12]:
# 고유값이 모두 고유값의 개수로 바뀐 것을 알 수 있다.
all_data['ps_ind_02_cat_count'].head()

0     309747
1    1079327
2      28259
3    1079327
4     309747
Name: ps_ind_02_cat_count, dtype: int64

In [13]:
# 정리
# 1. encoded_cat_matrix : one-hat encoded 명목형 피처
# 2. remaining_features : 명목형 피처와 calc 분류의 피처를 제외한 피처들 (+ num_missing 파생 피처)
# 3. cat_count_features : mix_ind를 포함한 명목형 피처의 고윳값별 개수 파생 피처

# 필요없는 피처 제거
# 1. 이전에 분류한 피처들 
drop_features = ['ps_ind_14', 'ps_ind_10_bin', 'ps_ind_11_bin',
                 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_car_14']

# remaining_features, cat_count_features에서 drop_features를 제거한 데이터
all_data_remaining = all_data[remaining_features+cat_count_features].drop(drop_features, axis=1)
# reamining_features와 cat_count_features를 합치고, drop_features를 제거한다.

In [14]:
# 피처 합치기
from scipy import sparse

all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data_remaining),
                               encoded_cat_matrix],
                              format='csr')

# all_data_remaining과 encoded_cat_matrix를 합침.
# hstack : 위 아래로 쌓아줌.

In [15]:
# 지금까지 한 것 정리
# 1. 명목형 피처에 원 핫 인코딩을 적용했습니다.
# 2. 데이터 하나당 가지고 있는 결측값 개수를 새로운 피처로 만들었습니다. - encoded_cat_matrix
# 3. 모든 ind 피처 값을 연결해서 새로운 명목형 피처를 만들었습니다. - mix_ind
# 4. 명목형 피처의 고윳값별 개수를 새로운 피처로 만들었습니다. - cat_count_features
# 5. 필요 없는 피처를 제거했습니다. (calc 분류의 피처, drop_features 제거)

In [16]:
# 데이터 나누기 - 이전과 동일함.

#from sklearn.utils import shuffle

# 전체 데이터를 훈련 데이터와 테스트 데이터로 다시 나눔.
num_train = len(train) #훈련 데이터 개수 -> 훈련 데이터 개수 만큼 나누기 위함.

# 셔플로 섞은 후, 나눠줌. -> hstack
#all_data_sprs = shuffle(all_data_sprs)

# 훈련 데이터와 테스트 데이터 나누기
X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

In [17]:
# 평가지표 작성 : 정규화된 지니계수
# 지니계수 : 지니계수가 작을수록 소득수준이 평등하고, 클수록 불평등함. -> 로렌츠 곡선 이용하여 계산함.
# 로렌츠 곡선 : 모든 경제인구를 소득 순서대로 나열한 후에 가로축은 인구 누적 비율, 새로축은 소득 누적 점유울로 설정함

# 정규화 지니계수 : 정규화 후 0에 가까울 수록 성능이 나쁘고, 1에 가까울수록 성능이 좋다.
# 위 : 예측 값에 대한 지니계수 : 예측값과 실제값으로 구한 지니계수
# 아래 : 예측이 완벽할 때의 지니계수 : 실제값과 실제값으로 구한 지니계수

import numpy as np

def eval_gini(y_true, y_pred):
    # 실제값과 예측값의 크기가 서로 같은 지 확인(값이 다르면 오류 발생)
    assert y_true.shape == y_pred.shape
    
    n_samples = y_true.shape[0] # 데이터 개수
    L_mid = np.linspace(1 / n_samples, 1, n_samples) # 대각선 값
    # n_samples 개수만큼 1/n_samples ~ 1 사이의 값으로 나눠줌.
    
    # 1) 예측값에 대한 지니계수
    pred_order = y_true[y_pred.argsort()] # y_pred 크기 순으로 y_true 값 정렬
    L_pred = np.cumsum(pred_order) / np.sum(pred_order) # 로렌츠 곡선
    # np.cumsum(pred_order) : pred_order의 누적합으로 이루어진 배열
    # 비율로 표현하기 위해서 np.sum(pred_order)로 나눔.
    G_pred = np.sum(L_mid - L_pred) # 예측값에 대한 지니계수
    
    # 2) 예측이 완벽할 때 지니계수
    true_order = y_true[y_true.argsort()] # y_true 크기 순으로 y_true 값 정렬
    L_true = np.cumsum(true_order) / np.sum(true_order) # 로렌츠 곡선
    G_true = np.sum(L_mid - L_true) # 예측값에 대한 지니계수
    
    # 정규화된 지니계수
    return G_pred / G_true

In [18]:
# ### 하이퍼 파라미터 최적화
# 베이지안 최적화 기법을 활용하여 하이퍼파라미터를 조정한다. -> 그리드서치보다 빠르고 효율적이며, 코드도 직관적이다.

# 1. 베이지안 최적화를 위한 데이터 셋을 만든다.
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# 8:2 비율로 훈련 데이터, 검증 데이터 분리함. (베이지안 최적화 수행용)
X_train, X_valid, y_train, y_valid = train_test_split(X, y,
                                                     test_size=0.2,
                                                     random_state=0)

# 베이지안 최적화용 데이터 셋 - 베이지안 최적화용 데이터셋 만듦.
bayes_dtrain = lgb.Dataset(X_train, y_train)
bayes_dvalid = lgb.Dataset(X_valid, y_valid)

In [26]:
# 하이퍼 파라미터 범위 설정 방법
# 1. 하이퍼파라미터 범위를 점점 좁히기. 0~1에서 베이지안 최적화 -> 0.5를 하이퍼 파라미터로 찾았다면 0.5 주변에서 베이지안 최적화 ...
# 2. 다른 상위권 캐글러가 설정한 하이퍼파라미터를 참고

# 베이지안 최적화를 위한 하이퍼파라미터 범위 - 2번 방법으로 직접 가져옴.
param_bounds = {'num_leaves': (30, 40),
                # 개별 트리가 가질 수 있는 최대 말단 노드 개수 (제한)
                # 트리의 복잡도를 결정하는 주요 파라미터로 값이 클수록 성능이 좋아질 수 있으나 과대적합 우려가 있음. (기본값 31)
                
                'lambda_l1': (0.7, 0.9),
                # L1 규제 조정값 - 값이 클수록 과대적합 방지 효과 있음. (기본 값 0)
                
                'lambda_l2': (0.9, 1),
                # L2 규제 조정값 - 값이 클수록 과대적합 방지 효과 있음. (기본 값 0)
                
                'feature_fraction': (0.6, 0.7),
                # 개별 트리를 훈련할 때, 사용하는 피처 샘플링 비율(0 ~ 1)
                # 전체 피처에서 얼마나 샘플링할지를 나타내는 비율
                # e.g. 0.7이면 개별 트리를 훈련할 때, 총 피처의 70%만 사용해 훈련함.
                # 값이 작을 수록 과대적합 방지 효과가 있음 (보통 0.6 ~ 1 사이의 값 사용함. 기본값 1)
                
                'bagging_fraction': (0.6, 0.9),
                # 개별 트리를 훈련할 때 사용할 데이터 샘플링 비율(0~1)
                # e.g. 0.5이면 전체 데이터의 50%를 사용하여 트리를 생성함.
                # 일반적으로 0.6 ~ 1 사이의 값을 사용. 더 작으면 샘플링할 데이터가 너무 적어짐. 
                # 기본값 1
                
                'min_child_samples': (6, 10),
                # 과대적합 방지를 위한 값
                # 말단 노드가 되기 위해 필요한 최소 데이터 개수
                # 기본값 20
                
                'min_child_weight': (10, 40)}
                # 과대적합 방지를 위한 값
                # 0 이상으로 설정할 수 있음.
                # 값이 클수록 과대적합 방지 효과가 있음.
                # 기본값 = 1e-3
                
# 베이지안 최적화를 수행하면 param_bounds의 하이퍼파라미터 범위를 순회함.
# 순회하면서 하이퍼파라미터 값을 적용하여 모델을 훈련하고, 훈련된 모델로 성능을 평가함.
# - 목적 : 최고 성능을 낸 하이퍼파라미터를 찾는 것. - 평가 지표 계산 함수는? 

# 값이 고정된 하이퍼 파라미터
fixed_params = {'objective' : 'binary', #이진분류 문제이므로,
                'learning_rate' : 0.005, # 학습률
                'bagging_freq' : 1, # 배깅 수행빈도 : 몇 번의 이터레이션마다 배깅을 수행할지 결정함.
                # c.f. 배깅(Bootstrap aggregating) : 중복을 허용한 랜덤 샘플링으로 훈련세트를 만들어서 학습.
                # 0 : 배깅을 수행하지 않음 (기본 값)
                # 1 : 전달 시 매 이터레이션마다 트리가 새로운 샘플링 데이터로 학습함.
                'force_row_wise' : True, # 경고 문구 제거용
                'random_state' : 46} # 랜덤 시드값 (코드를 반복 실행해도 같은 결과가 나오게 지정하는 값)
                # c.f. 경우에 따라서(보통 디버깅 등을 위해 ) 동일한 순서로 난수를 발생시켜야 할 경우가 있으므로, 이를 위한 값.

In [27]:
# LightGBM으로 넘겨주기 위한 용도 : gini() 함수
def gini(preds, dtrain):
    labels = dtrain.get_label()  # 데이터값의 타깃값을 반환함.
    return 'gini', eval_gini(labels, preds), True
    # 평가지표 이름, 평가점수, 평가 점수가 높을수록 좋은지 여부 (여기서는 지니계수가 높을수록 좋으므로)

In [28]:
# 베이지안 최적화용 평가지표 계산 함수 -> 이 함수로 지니계수를 계산해 최적의 하이퍼파라미터를 찾는다.
# LightGBM의 하이퍼파라미터 7개를 인수로 받고 지니계수를 반환한다.

def eval_function(num_leaves, lambda_l1, lambda_l2, feature_fraction,
                  bagging_fraction, min_child_samples, min_child_weight):
    '''최적화하려는 평가지표(지니계수) 계산 함수'''
    
    # 베이지안 최적화를 수행할 하이퍼파라미터
    # 인수로 받은 하이퍼파라미터 값(범위)를 그대로 대입함.
    # int(round(@)) : 실수형을 정수형으로 바꿔줌. - num_leaves, min_child_samples는 정수여야함.
    # 왜 필요하나? 베이지안 최적화하면 하이퍼파라미터 지정 범위 내 실수값을 탐색하므로 전달하는 값을 정수형으로 바꿔주기 위함.
    params = {'num_leaves' : int(round(num_leaves)),
              'lambda_l1' : lambda_l1,
              'lambda_l2' : lambda_l2,
              'feature_fraction' : feature_fraction,
              'bagging_fraction' : bagging_fraction,
              'min_child_samples' : int(round(min_child_samples)),
              'min_child_weight' : min_child_weight,              
              'feature_pre_filter' : False}
    
    # 고정된 하이퍼 파라미터도 추가 - 우수한 캐글 신의 도움.
    # params는 딕셔너리 타입이므로, update() 함수로 추가해줌.
    params.update(fixed_params)
    
    print('하이퍼 파라미터:', params)
    
    # LightGBM 모델 훈련 - 위에서 정의한 하이퍼 파라미터 정의 사용.
    lgb_model = lgb.train(params=params, #훈련용 하이퍼 파라미터
                          train_set=bayes_dtrain, # 베이지안 훈련 데이터 셋
                          num_boost_round=2500, #부스팅 반복 횟수
                          # 클수록 성능이 좋아히지나 과대적합 우려.
                          # 작을수록 반복횟수가 줄어들어 훈련시간이 짧아짐.
                          # 일반적으로 num_boost_round를 늘리면 learning_rate를 줄여야함.
                          valid_sets=bayes_dvalid, #성능 평가용 검증 데이터 셋
                          feval=gini, #검증용 평가지표
                          early_stopping_rounds=300, #조기종료 조건
                          # num_boost_round 만큼 훈련을 반복하는데, 매 이터레이션 마다 eval로 평가시 성능이 연속적으로 좋아지지않으면 중단.
                          verbose_eval=100) # 100번째마다 점수 출력 - 출력값이 많아지는 걸 방지.
    
    
    # 검증 데이터로 예측 수행 - K개 만큼 나옴.
    preds = lgb_model.predict(X_valid)
    
    # 지니계수 계산 - 예측값과 검증 데이터 라벨(타겟값)을 이용하여 지니계수를 계산함.
    gini_score = eval_gini(y_valid, preds)
    print(f'지니계수 : {gini_score}\n')
    
    # 지니계수 반환.
    return gini_score

In [33]:
# 최적화 수행
# 하이퍼 파라미터를 추정하는 과정
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function, # 평가지표 계산 함수
                                 pbounds=param_bounds, # 하이퍼파라미터 범위
                                 random_state=0)

# 베이지안 최적화 수행
# maximize() 메서드를 호출하여 베이지안 최적화를 수행한다.
optimizer.maximize(init_points=3, n_iter=6)

# init_points : 무작위로 하이퍼파라미터를 탐색하는 횟수
# n_iter : 베이지안 최적화 반복 횟수
# 반복하는 횟수 : init_points + n_iter e.g. 3+6 = 9

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
하이퍼파라미터: {'num_leaves': 34, 'lambda_l1': 0.8205526752143287, 'lambda_l2': 0.9544883182996897, 'feature_fraction': 0.6715189366372419, 'bagging_fraction': 0.7646440511781974, 'min_child_samples': 8, 'min_child_weight': 29.376823391999682, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 46}


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1554
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.28466512416542983

| 1         | 0.2847    | 0.7646    | 0.6715    | 0.8206    | 0.9545    | 7.695     | 29.38     | 34.38     |
하이퍼파라미터: {'num_leaves': 39, 'lambda_l1': 0.7766883037651555, 'lambda_l2': 0.9791725038082665, 'feature_fraction': 0.6963662760501029, 'bagging_fraction': 0.867531900234624, 'min_child_samples': 8, 'min_child_weight': 27.04133683281797, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 46}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1554
[LightGBM] [Info] Number of data points in the

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 1554
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.28472499359959924

| 4         | 0.2847    | 0.7636    | 0.6332    | 0.7182    | 0.9381    | 8.348     | 38.16     | 39.59     |
하이퍼파라미터: {'num_leaves': 37, 'lambda_l1': 0.7738449330497988, 'lambda_l2': 0.9032695189818599, 'feature_fraction': 0.6606341064409726, 'bagging_fraction': 0.7666713964943057, 'min_child_samples': 9, 'min_child_weight': 29.306172421380474, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 46}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786
[LightGBM] [Info] Total Bins 1554
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [bi

...

[LightGBM] [Info] Total Bins 1554
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.28510249128741233

| 7         | 0.2851    | 0.7237    | 0.6556    | 0.8621    | 0.9005    | 8.223     | 16.8      | 36.38     |
하이퍼파라미터: {'num_leaves': 32, 'lambda_l1': 0.829932662510154, 'lambda_l2': 0.9309382272206734, 'feature_fraction': 0.6952194054160209, 'bagging_fraction': 0.6675659367709144, 'min_child_samples': 8, 'min_child_weight': 28.604068940720285, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 46}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 1554
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.28568536709429204

| 8         | 0.2857    | 0.6676    | 0.6952    | 0.8299    | 0.9309    | 7.88      | 28.6      | 31.94     |
하이퍼파라미터: {'num_leaves': 37, 'lambda_l1': 0.8069422448218244, 'lambda_l2': 0.9757043485406661, 'feature_fraction': 0.6837697894740693, 'bagging_fraction': 0.7295929036037181, 'min_child_samples': 8, 'min_child_weight': 34.57982411404511, 'feature_pre_filter': False, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 46}
[LightGBM] [Info] Number of positive: 17383, number of negative: 458786


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 1554
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 217
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036506 -> initscore=-3.273091
[LightGBM] [Info] Start training from score -3.273091
지니계수 : 0.28529747737312666

| 9         | 0.2853    | 0.7296    | 0.6838    | 0.8069    | 0.9757    | 8.408     | 34.58     | 36.57     |


In [34]:
# 베이지안 최적화 결과 확인
# 지니계수가 최대가 되는 최적의 하이퍼 파라미터를 얻을 수 있음.

# 평가점수(지니계수)가 최대일 때 하이퍼파라미터
max_params = optimizer.max['params']

print(max_params)
print()

# nums_leaves와 min_child_samples는 정수형 하이퍼 파라미터 이므로 정수형으로 변환하여 다시 저장한다.
max_params['num_leaves'] = int(round(max_params['num_leaves']))
max_params['min_child_samples'] = int(round(max_params['min_child_samples']))

# max_params에 fixed_params를 추가한다. - 고정 파라미터 추가
max_params.update(fixed_params)

# 찐 최종 파라미터 - 그리드 서치와는 달리 베이지안 최적화는 최적 예측기(최적 하이퍼파라미터 값으로 훈련된 모델)를 제공하지 않는다! -> 훈련 필요
print(max_params)

{'bagging_fraction': 0.6213108174593661, 'feature_fraction': 0.608712929970154, 'lambda_l1': 0.7040436794880651, 'lambda_l2': 0.9832619845547939, 'min_child_samples': 9.112627003799401, 'min_child_weight': 36.10036444740457, 'num_leaves': 39.78618342232764}

{'bagging_fraction': 0.6213108174593661, 'feature_fraction': 0.608712929970154, 'lambda_l1': 0.7040436794880651, 'lambda_l2': 0.9832619845547939, 'min_child_samples': 9, 'min_child_weight': 36.10036444740457, 'num_leaves': 40, 'objective': 'binary', 'learning_rate': 0.005, 'bagging_freq': 1, 'force_row_wise': True, 'random_state': 46}


In [35]:
# OOF 구현
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
# 타깃값이 불균형 하므로, K폴드가 아닌 층화 K 폴드 방식을 사용함.
# 층화 K 폴드 방식 : 타깃값이 균등하게 배치되게 폴드를 나누는 방식. -> 하나하나 동일한 비율로 나눠주는 것
# 훈련데이터를 섞어줌. 특정 패턴 데이터가 일부 폴드에만 몰려있으면 모델 성능이 과적합 되므로.
folds = StratifiedKFold(n_splits = 5, shuffle = True, random_state=42)

In [36]:
# OOF 방식으로 훈련된 모델로 검증 데이터 타겟값을 예측한 확률을 담을 1차원 배열
oof_val_preds = np.zeros(X.shape[0]) # 훈련 데이터 크기만큼 설정. 폴더로 나눠도 어차피 모두 합하면 K개임.

# OOF 방식으로 훈련된 모델로 테스트 데이터 타겟값을 예측한 확률을 담을 1차원 배열
oof_test_preds = np.zeros(X_test.shape[0]) # 테스트 데이터를 통해 예측한 확률값을 저장함.

In [37]:
# LightGBM 모델 훈련 + OOF 예측
import lightgbm as lgb

# OOF 방식으로 모델을 훈련, 검증, 예측
for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    #folds.split : 데이터를 k개로 나눔
    
    #각 폴드를 구분하는 문구 출력
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    # 훈련용 데이터, 검증용 데이터 설정
    X_train, y_train = X[train_idx], y[train_idx] #훈련용 데이터
    X_valid, y_valid = X[valid_idx], y[valid_idx] #검증용 데이터
    
    # LightGM 전용 데이터셋 생성
    dtrain = lgb.Dataset(X_train, y_train) # LightGBM 전용 훈련 데이터 셋
    dvalid = lgb.Dataset(X_valid, y_valid) # LightGBM 전용 검증 데이터 셋
    
    # LightGBM 모델 훈련
    lgb_model = lgb.train(params=max_params, #훈련용 하이퍼 파라미터 -> 최적 하이퍼 파라미터로 변경
                          train_set=dtrain, #훈련 데이터 셋
                          num_boost_round=1000, #부스팅 반복 횟수
                          # 클수록 성능이 좋아히지나 과대적합 우려.
                          # 작을수록 반복횟수가 줄어들어 훈련시간이 짧아짐.
                          # 일반적으로 num_boost_round를 늘리면 learning_rate를 줄여야함.
                          valid_sets=dvalid, #성능 평가용 검증 데이터 셋
                          feval=gini, #검증용 평가지표
                          early_stopping_rounds=100, #조기종료 조건
                          # num_boost_round 만큼 훈련을 반복하는데, 매 이터레이션 마다 eval로 평가시 성능이 연속적으로 좋아지지않으면 중단.
                          verbose_eval=100) # 100번째마다 점수 출력 - 출력값이 많아지는 걸 방지.

    # 테스트 데이터를 활용해 OOF 예측 - folds.n_splits로 나눈 이유는 확률을 구해야하므로.
     # predict() 호출시 훈련 단계에서 최고 성능을 낸 반복 횟수로 예측한다.
    # e.g. 500번째 이터레이션에서 지니계수가 최고라면 500번째꺼 사용함.(default)
    # 주의할점은 X_test, X-valid 그대로 사용해야한다는 점 lgb.Dataset()변환 X
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    
    # 모델 성능 평가를 위한 검증 데이터 타겟값 예측
    # valid_idx에 해당하는 값만 검증 데이터 예측 확률로 업데이트해줌.
    # 폴드가 5번 반복되면 off_val_preds 내 모든 값이 검증 데이터 예측확률로 업데이트됨.
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)
    
    # 검증 데이터 예측 확률에 대한 정규화 지니계수
    #
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} 지니계수: {gini_score}\n')

######################################## 폴드 1 / 폴드 5 ########################################


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1554
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 216
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.154255	valid_0's gini: 0.273723
[200]	valid_0's binary_logloss: 0.15317	valid_0's gini: 0.279059
[300]	valid_0's binary_logloss: 0.152573	valid_0's gini: 0.28264
[400]	valid_0's binary_logloss: 0.152213	valid_0's gini: 0.285212
[500]	valid_0's binary_logloss: 0.151973	valid_0's gini: 0.28789
[600]	valid_0's binary_logloss: 0.151807	valid_0's gini: 0.289982
[700]	valid_0's binary_logloss: 0.151676	valid_0's gini: 0.292139
[800]	valid_0's binary_logloss: 0.15158	valid_0's gini: 0.293764
[900]	valid_0's binary_logloss: 0.151521	valid_0's gini: 0.294574
[1000]	valid_

In [39]:
# 훈련 종료. 결과.
# 검증 데이터로 예측한 확률을 실제 타깃값과 비교하여 지니계수를 출력함.
print('OOF 검증 데이터 지니계수 : ', eval_gini(y, oof_val_preds))

OOF 검증 데이터 지니계수 :  0.286007908703013


In [40]:
# 예측 및 결과 제출
submission['target'] = oof_test_preds
submission.to_csv('submission.csv')